Get your openai api key from settings: https://platform.openai.com/account/api-keys

In [1]:
%pip install openai --upgrade
%pip install pydub simpleaudio
%pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\mattj\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\mattj\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\mattj\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
# Get the openai secret key
import getpass
from openai import OpenAI

secret_key = getpass.getpass("Please enter your openai key: ")
import os
os.environ["OPENAI_API_KEY"]= secret_key


In [42]:

# OpenAI set up:
client = OpenAI()
client.api_key = secret_key
# Set up OpenAI client with the provided API key

def complete(prompt, stop=None):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
             {
                "role": "system",
                "content": "You are a real-time voice chat assistant for Spanish language learning, specializing in Mexican Spanish with a natural Mexican accent."
            },
            {
            "role": "user",
            "content": prompt
            }
        ],
        stop=stop
    )
    return response.choices[0].message.content

complete("is this working?")

'¡Hola! Sí, estoy aquí. ¿Cómo puedo ayudarte con tu aprendizaje del español hoy?'

In [3]:
from pathlib import Path
from openai import OpenAI
#client = OpenAI()
filename1 = "speech.mp3"

speech_file_path = Path.cwd() / filename1
response = client.audio.speech.create(
  model="tts-1",
  voice="onyx",
  input="a tu orden"
)

response.stream_to_file(speech_file_path)

C:\Users\mattj\AppData\Local\Temp\ipykernel_3080\3526707331.py:13: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [7]:
from pydub import AudioSegment
import simpleaudio as sa
from pathlib import Path


# Path to the MP3 file
filename1 = "intro.mp3"

#speech_file_path = Path("speech.mp3")
# Load and play the audio file
speech_file_path = Path.cwd() / filename1
audio = AudioSegment.from_mp3(speech_file_path)
play_obj = sa.play_buffer(
    audio.raw_data,
    num_channels=audio.channels,
    bytes_per_sample=audio.sample_width,
    sample_rate=audio.frame_rate
)
play_obj.wait_done()  # Wait until playback is finished


In [ ]:


''' 
REcord user audio (no idea how to start and stop this)(for testing should i just have a set amount of time?)

''' 


In [ ]:
'''
#transcribe to text
audio_file = open("/path/to/file/speech.mp3", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file, 
  response_format="text"
)
print(transcription.text)

'''

In [62]:
myText = "Yo ir al tienda para comprar manzanas y platanos. Cuando llegué, vi muchas frutas y verduras. El cajero me dijo que el precio son muy alto, pero no me importa. Después, yo regresar a casa y preparé un plato grande para mi familia. Todos nos gusta comer juntos."
#myText = "Hola, como estas"
myPrompt1 = """
You will receive a paragraph written in Spanish. Your task is to identify and correct major grammatical errors while adhering to the following instructions:

1. Ignore gender errors.
2. If a mistake is made but then corrected within the same paragraph, do not include that correction (e.g., "yo comí" followed by "yo comer").
3. Ignore typing mistakes or missing accents; these are not considered errors.
4. Ignore minor clarity issues.
5. If there are no errors worth mentioning, output "No Errors."

Your output should consist only of the correct Spanish for each identified error, separated by periods. Do not provide any additional commentary or explanations. Aim for responses of 4-5 words unless more is needed.

Example:

Input: "Nosotros va a la playa mañana. Ellos trae sus juguetes."

Output: "Nosotros vamos. Ellos traen."
"""
 
myPrompt2 = """ 
You will receive a paragraph written in Spanish. Your task is to identify and correct one major grammatical error while adhering to the following instructions:
Ignore gender errors.
If a mistake is made but then corrected within the same paragraph, do not include that correction (e.g., "yo comí" followed by "yo comer").
Ignore typing mistakes or missing accents; these are not considered errors.
Ignore minor clarity issues.
If there are no significant errors, output "No Errors."
Your output should consist of three sentences:

The original incorrect sentence, abreviated to show what was said, but not include extraneous detail.
The corrected version with the corrected word in all caps.
If no significant error is found, just write "No Errors."
Example:

Input: Nosotros va a la playa mañana para solear. Ellos trae sus juguetes.

Output: Nosotros va a la playa mañana. Nosotros VAMOS a la playa mañana.
"""

In [63]:
gptResponse = complete(f"{myPrompt2}\n\n{myText}")
print(gptResponse)


Yo ir al tienda para comprar manzanas y platanos. Yo FUI al tienda para comprar manzanas y platanos.


In [44]:
import nltk
#nltk.download('punkt_tab')  # Download the punkt tokenizer if you haven't already

def split_paragraph_into_sentences(paragraph):
    sentences = nltk.tokenize.sent_tokenize(paragraph)
    return sentences

# Example usage
responseSentenceSplit = split_paragraph_into_sentences(gptResponse)
print(responseSentenceSplit)

['"Voy a la tienda.', 'El precio es muy alto.', 'Después, regresé a casa.', 'Todos nos gusta."']


In [ ]:
#convert individual errors into audio
if gptResponse != "No Errors.":
    for i, sentence in enumerate(responseSentenceSplit, start=1):
        filename = f"error{i}.mp3"
        print(f"Filename: {filename} - Sentence: {sentence}")
        speech_file_path = Path.cwd() / filename
        response = client.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=sentence
        )

        response.stream_to_file(speech_file_path)

    

Filename: error1.mp3 - Sentence: "Voy a la tienda.


C:\Users\mattj\AppData\Local\Temp\ipykernel_3080\1777402755.py:13: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


Filename: error2.mp3 - Sentence: El precio es muy alto.
Filename: error3.mp3 - Sentence: Después, regresé a casa.
Filename: error4.mp3 - Sentence: Todos nos gusta."


In [49]:
from pydub import AudioSegment
mergedFilename = "MergedErrors.mp3"
# List of filenames to be merged
filenames = [f"error{i}.mp3" for i in range(1, len(responseSentenceSplit) + 1)]

# Start with an empty AudioSegment
full_audio = AudioSegment.empty()

# Set the duration of silence (in milliseconds)
pause_duration =200  # 500 milliseconds = 0.5 seconds

# Iterate through each file and add it to full_audio
for filename in filenames:
    audio = AudioSegment.from_mp3(filename)
    full_audio += audio + AudioSegment.silent(duration=pause_duration) # Concatenate each audio file

# Export the combined audio to Full.mp3
full_audio.export(mergedFilename, format="mp3")

# play audio back
audio_file_path = Path.cwd() / mergedFilename
audio = AudioSegment.from_mp3(audio_file_path)
play_obj = sa.play_buffer(
    audio.raw_data,
    num_channels=audio.channels,
    bytes_per_sample=audio.sample_width,
    sample_rate=audio.frame_rate
)
play_obj.wait_done()  # Wait until playback is finished

In [64]:
#convert errors into audio
if gptResponse != "No Errors.":
    filename1 = "speech.mp3"
    speech_file_path = Path.cwd() / filename1
    response = client.audio.speech.create(
    model="tts-1",
    voice="onyx",
    input=gptResponse
    )

    response.stream_to_file(speech_file_path)

    # play audio back
    audio = AudioSegment.from_mp3(speech_file_path)
    play_obj = sa.play_buffer(
        audio.raw_data,
        num_channels=audio.channels,
        bytes_per_sample=audio.sample_width,
        sample_rate=audio.frame_rate
    )
    play_obj.wait_done()  # Wait until playback is finished

C:\Users\mattj\AppData\Local\Temp\ipykernel_3080\3506220433.py:11: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [ ]:
''' 
if no corrections are needed, play 'cuentame.mp3' 
else:
build a list of corrections
ignore anything that's just a gender correction
trim the correction down to the minimal amount of speaking
generate audio for each correction (will the audio suck if it's not one long sentence and instead is multiple ones strung together?)
    or, concat all the corrections and audio generate

play audio
'''

In [ ]:

''' 
if the last words said were  
nevermind - just move one
repite - auto replay everything that was just said and ignore 
repite (algo) figure out what exactly i want repeated and repeat that audio slowly (consider slowing down the playback)

Listen to audio
I respond normally - one by one, go through the things that were said to me and check if i said them correctly back. 
    if yes, then move to the next item,
    if no, add that item back to the audio output
    if all yes, then continue conversation
    otherwise, concat audio again and play new audio

'''